In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.insert(0, "../../code-previous")

In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import sklearn.feature_extraction as skfeatures
import time
import os

import operator

import string
import swifter
import nltk
stopwords = nltk.corpus.stopwords.words('english')


import plotly
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='sreejith_s', api_key='rcet0rTqFCc3WeOzWTjv')



from sklearn.feature_extraction.text import CountVectorizer


from unicodedata import category
from tqdm import tqdm
tqdm.pandas(desc="Progress")

import seaborn as sns
sns.set_style('white')
import matplotlib.pylab as plt

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
data_path = "/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/all_discipline_data.pql"

#data_path = "/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/physics.pql"
data = pd.read_pickle(data_path)

In [5]:
physics_labels = ['Physics, Applied',
                  'Physics, Fluids & Plasmas',
                  'Physics, Atomic, Molecular & Chemical',
                  'Physics, Multidisciplinary',
                  'Physics, Condensed Matter',
                  'Physics, Nuclear',
                  'Physics, Particles & Fields',
                  'Physics, Mathematical']

In [6]:
physics_data = data[data["Label"].isin(physics_labels)]
article_df = None

In [7]:
#physics_applied_phy = physics_data[physics_data.Label == "Physics, Nuclear"]
physics_applied_phy = physics_data.sort_values(by="PubYear")
vectorizer1 = CountVectorizer()
vectorized_words1 = vectorizer1.fit_transform(physics_applied_phy.title_without_stopwords)

In [8]:
chemistry_labels = ['Chemistry, Applied',
                    'Chemistry, Multidisciplinary',
                    'Chemistry, Analytical',
                    'Chemistry, Inorganic & Nuclear',
                    'Chemistry, Organic',
                    'Chemistry, Physical',
                    'Electrochemistry']

In [9]:
chemistry_data = data[data["Label"].isin(chemistry_labels)]

In [10]:
#physics_applied_phy = physics_data[physics_data.Label == "Physics, Nuclear"]
chemistry_data = chemistry_data.sort_values(by="PubYear")
vectorizer_chem = CountVectorizer()
chemistry_data = vectorizer_chem.fit_transform(chemistry_data.title_without_stopwords)